"Meta": one level up

"Metaprogramming": programming a program

Talk about Julia code within Julia:

cf:  Alan said "My name is Alan"

In [2]:
ex = 
quote
    z = x + y
end

quote  # In[2], line 3:
    z = x + y
end

In [3]:
ex  # unevaluated piece of code

quote  # In[2], line 3:
    z = x + y
end

In [4]:
z = x + y

LoadError: LoadError: UndefVarError: x not defined
while loading In[4], in expression starting on line 1

In [5]:
ex

quote  # In[2], line 3:
    z = x + y
end

In [6]:
typeof(ex)

Expr

In [7]:
fieldnames(ex)

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

Ignore `typ`

In [8]:
ex.head

:block

In [9]:
ex.args

2-element Array{Any,1}:
 :( # In[2], line 3:)
 :(z = x + y)        

In [10]:
ex.args[1]

:( # In[2], line 3:)

In [11]:
typeof(ans)

Expr

In [12]:
ex.args[2]

:(z = x + y)

In [13]:
typeof(ans)

Expr

In [14]:
ex = :( z = x + y )  # :(  ...  )  also makes an unevaluated code object

:(z = x + y)

In [15]:
typeof(ex)

Expr

In [16]:
ex.head

:(=)

In [17]:
typeof(ex.head)

Symbol

Symbol is the smallest atom in an expression

In [20]:
:(=)

:(=)

In [21]:
ex.args

2-element Array{Any,1}:
 :z      
 :(x + y)

In [22]:
typeof(:z)

Symbol

In [23]:
ex2 = :(x + y)

:(x + y)

In [24]:
typeof(ex2)

Expr

In [25]:
ex = :(x + y * z)

:(x + y * z)

In [27]:
ex.head

:call

In [28]:
ex.args

3-element Array{Any,1}:
 :+      
 :x      
 :(y * z)

In [29]:
ex2 = :( (x + y) * z)

:((x + y) * z)

In [30]:
ex2.args

3-element Array{Any,1}:
 :*      
 :(x + y)
 :z      

In [31]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol y
        3: Symbol z
      typ: Any
  typ: Any


In [32]:
Meta.show_sexpr(ex)

(:call, :+, :x, (:call, :*, :y, :z))

## Manipulate expression

In [33]:
ex = :(3x + 2x^2)

:(3x + 2 * x ^ 2)

I want to replace all the `x`s by (`x+1`)

In [34]:
ex.args

3-element Array{Any,1}:
 :+          
 :(3x)       
 :(2 * x ^ 2)

In [36]:
ex.args[2].args

3-element Array{Any,1}:
  :*
 3  
  :x

In [37]:
ex.args[2].args[3]

:x

In [38]:
ex.args[2].args[3] = :(x+1)

:(x + 1)

In [39]:
ex

:(3 * (x + 1) + 2 * x ^ 2)

In [40]:
eval(ex)

LoadError: LoadError: UndefVarError: x not defined
while loading In[40], in expression starting on line 1

In [41]:
x = 10
eval(ex)

233

In [42]:
s = "3x + 3x^2"

"3x + 3x^2"

Turn this string into an Expr:

In [43]:
ex = parse(s)

:(3x + 3 * x ^ 2)

In [44]:
x = 10
eval(ex)

330

What I shouldn't do:

In [45]:
s2 = replace(s, "x", "x+1")

"3x+1 + 3x+1^2"

In [46]:
s2 = replace(s, "x", "(x+1)")

"3(x+1) + 3(x+1)^2"

In [47]:
ex

:(3x + 3 * x ^ 2)

## Walk through tree

Use recursive function to walk through the tree

In [50]:
ex = :(x + 3x^2)

:(x + 3 * x ^ 2)

In [51]:
ex.args

3-element Array{Any,1}:
 :+          
 :x          
 :(3 * x ^ 2)

In [61]:
function traverse!(ex::Expr)
    
    for i in 1:length(ex.args)
        
        if isa(ex.args[i], Expr)  # if this object is of type Expr; or:  if typeof(ex.args[i]) == Expr
            traverse!(ex.args[i])
            
        elseif ex.args[i] == :x
            ex.args[i] = :(x+1)
        end
        
    end
end

traverse! (generic function with 1 method)

In [53]:
traverse!(ex)

In [54]:
ex

:((x + 1) + 3 * x ^ 2)

In [62]:
ex = :(x + 3x^2)

:(x + 3 * x ^ 2)

In [63]:
traverse!(ex)

In [64]:
ex

:((x + 1) + 3 * (x + 1) ^ 2)

In [65]:
ex = :(3x * (x^2 + (x-1) / (x+1)^3))

:((3x) * (x ^ 2 + (x - 1) / (x + 1) ^ 3))

In [66]:
traverse!(ex)

In [67]:
ex

:((3 * (x + 1)) * ((x + 1) ^ 2 + ((x + 1) - 1) / ((x + 1) + 1) ^ 3))

In [76]:
function traverse!(ex::Expr, replace_what, replace_with)
    
    for i in 1:length(ex.args)
        
        if isa(ex.args[i], Expr)  # if this object is of type Expr; or:  if typeof(ex.args[i]) == Expr
            traverse!(ex.args[i], replace_what, replace_with)
            
            elseif ex.args[i] == replace_what
                ex.args[i] = replace_with
        end
        
    end
end

traverse! (generic function with 2 methods)

In [77]:
methods(traverse!)

# 2 methods for generic function "traverse!":
traverse!(ex::Expr) at In[61]:3
traverse!(ex::Expr, replace_what, replace_with) at In[76]:3

In [78]:
ex = :(3x+3x^2)
traverse!(ex, :x, :(f(x+1)) )

In [79]:
ex

:(3 * f(x + 1) + 3 * f(x + 1) ^ 2)

quote  # In[81], line 2:
    type Hello # In[81], line 3:
    end
end

Want to replace "numeric literals" (3 or 4.5) by f(3):

In [93]:
v = [1, 2, 3]
map(sin, v)

3-element Array{Float64,1}:
 0.841471
 0.909297
 0.14112 

In [117]:
function traverse(ex::Expr)
    new_ex = quote end  # empty expression
    new_ex.head = ex.head
    new_ex.args = map(traverse, ex.args)
    
    return new_ex
end

traverse(ex::Number) = :(f($ex))

traverse(ex::Symbol) = ex

traverse(ex) = ex

traverse (generic function with 4 methods)

In [95]:
ex = :(3x + 4.5)

:(3x + 4.5)

In [96]:
ex2 = traverse(ex)

quote 
    +
    begin 
        *
        f(3)
        x
    end
    f(4.5)
end

In [90]:
Meta.show_sexpr(ex2)

(:block,
  :+,
  (:block,
    :*,
    (:call, :f, 3),
    :x
  ),
  (:call, :f, 4)
)

## Macros

In [97]:
@time sin(10)

  0.003258 seconds (142 allocations: 8.469 KB)


-0.5440211108893698

`@` is the signal that it's a "macro", i.e. a meta-function.

A function that takes as its argument a piece of code, manipulates it, and returns the resulting piece of code.

In [98]:
macroexpand(:(@time sin(10)))

quote  # util.jl, line 182:
    local #6#stats = (Base.gc_num)() # util.jl, line 183:
    local #8#elapsedtime = (Base.time_ns)() # util.jl, line 184:
    local #7#val = sin(10) # util.jl, line 185:
    #8#elapsedtime = (Base.time_ns)() - #8#elapsedtime # util.jl, line 186:
    local #9#diff = (Base.GC_Diff)((Base.gc_num)(),#6#stats) # util.jl, line 187:
    (Base.time_print)(#8#elapsedtime,#9#diff.allocd,#9#diff.total_time,(Base.gc_alloc_count)(#9#diff)) # util.jl, line 189:
    #7#val
end

In [99]:
using MacroTools

In [100]:
@expand @time sin(10)

quote  # util.jl, line 182:
    local camel = (Base.gc_num)() # util.jl, line 183:
    local cat = (Base.time_ns)() # util.jl, line 184:
    local goosander = sin(10) # util.jl, line 185:
    cat = (Base.time_ns)() - cat # util.jl, line 186:
    local hyena = (Base.GC_Diff)((Base.gc_num)(),camel) # util.jl, line 187:
    (Base.time_print)(cat,hyena.allocd,hyena.total_time,(Base.gc_alloc_count)(hyena)) # util.jl, line 189:
    goosander
end

In [101]:
function mytime(x)
    @show x
end

mytime (generic function with 1 method)

In [102]:
mytime(sin(10))

x = -0.5440211108893698


-0.5440211108893698

A macro takes an expression as argument:

In [103]:
macro simple(ex)
    show(ex)
    return nothing
end

@simple (macro with 1 method)

In [104]:
@simple 3x

:(3x)

In [105]:
@simple sin(10)

:(sin(10))

In [106]:
ex = :(3x)

:(3x)

In [118]:
traverse(ex)

:(f(3) * x)

In [108]:
typeof(ans)

Expr

In [109]:
macro traverse(ex)
    new_ex = traverse(ex)   # calls the **function** traverse

    return new_ex
end

@traverse (macro with 1 method)

In [119]:
@traverse 3x

40

In [111]:
f(x) = x+1

f (generic function with 1 method)

In [112]:
x = 10

10

In [113]:
@traverse 3x

10

In [114]:
f(3)

4

In [115]:
traverse(:(3x))

quote 
    *
    f(3)
    x
end

In [116]:
eval(ans)

10

## Uses of macros

Domain-specific languages (DSL):

- JuMP: Mathematical optimization -- make it easier to express the problem
- Steven Johnson: Evaluate numerical functions:

In [121]:
apropos("horner")

Base.Math.@evalpoly


In [122]:
?Base.Math.@evalpoly

```
@evalpoly(z, c...)
```

Evaluate the polynomial $\sum_k c[k] z^{k-1}$ for the coefficients `c[1]`, `c[2]`, ...; that is, the coefficients are given in ascending order by power of `z`.  This macro expands to efficient inline code that uses either Horner's method or, for complex `z`, a more efficient Goertzel-like algorithm.


In [125]:
f(x) = x + x^2 - 3x^3

f (generic function with 1 method)

In [128]:
@expand Base.Math.@evalpoly(10, 0, 1, 1, -3)  # inlined

:(let llama = 10 # math.jl, line 111:
        if (Base.Math.isa)(llama,Base.Math.Complex)
            wombat = (Base.Math.real)(llama)
            ape = (Base.Math.imag)(llama)
            crocodile = wombat + wombat
            echidna = (Base.Math.muladd)(wombat,wombat,ape * ape)
            peafowl = -3
            prairiedog = (Base.Math.muladd)(crocodile,peafowl,1)
            elephant = (Base.Math.muladd)(crocodile,prairiedog,1 - echidna * peafowl)
            (Base.Math.muladd)(elephant,llama,0 - echidna * prairiedog)
        else 
            ant = llama
            (Base.Math.muladd)(ant,(Base.Math.muladd)(ant,(Base.Math.muladd)(ant,-3,1),1),0)
        end
    end)

In [126]:
f(x) = x * (1 + x * (-3x + 1) )

-2890

In [ ]:
x * (x * (-3x + 1) + 1) + 0

In [129]:
@edit +(3.4, 5.4)

In [130]:
function mytime(f, args...)
    tic()
    f(args...)
    toc()
end

mytime (generic function with 2 methods)

In [131]:
mytime(sin, 10)

elapsed time: 2.0184e-5 seconds


2.0184e-5

In [132]:
3

3

In [133]:
typeof(ans)

Int64

In [134]:
x = 3

3

In [135]:
typeof(x)

Int64

In [136]:
v = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [139]:
function f(x)
    x = x + 1
    y = x
    return y
end

f (generic function with 1 method)

In [140]:
x = 3

3

In [141]:
f(x)

4

In [142]:
x

3

In [143]:
function f(x)
    x[1] = 10
end

f (generic function with 1 method)

In [144]:
x = [1, 2]

2-element Array{Int64,1}:
 1
 2

In [145]:
f(x)

10

In [146]:
x

2-element Array{Int64,1}:
 10
  2

In [147]:
function g(x)
    x = zeros(x)
    @show x
end

g (generic function with 1 method)

In [148]:
x

2-element Array{Int64,1}:
 10
  2

In [149]:
g(x)

x = [0,0]


2-element Array{Int64,1}:
 0
 0

In [150]:
x

2-element Array{Int64,1}:
 10
  2